In [1]:
import pandas as pd
import numpy as np
import cv2
import os
import re

from PIL import Image

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import torch
import torchvision

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler

from matplotlib import pyplot as plt

In [2]:
with open('./dataset/data.txt', 'r') as f:
    datas = f.read().split('\n')

In [3]:
class WheatDataset(Dataset):

    def __init__(self, datas, transforms=None):
        super().__init__()

        # self.image_ids = dataframe['image_id'].unique()
        self.datas = datas
        # self.image_dir = image_dir
        self.transforms = transforms

    def __getitem__(self, index: int):
        # image_id = self.image_ids[index]
        # records = self.df[self.df['image_id'] == image_id]
        data = self.datas[index]
        path = data.split()[0]
        bboxs = data.split()[1:]

        image = cv2.imread("../trainval" + path, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0

        labels = []
        boxes = []
        for bbox in bboxs:
            boxes.append(list(map(int, bbox.split(',')))[:4])
            labels.append(list(map(int, bbox.split(',')))[4])
            
        boxes = np.array(boxes, dtype=np.float32)
        labels = torch.from_numpy(np.array(labels, dtype=np.int64))

        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        area = torch.as_tensor(area, dtype=torch.float32)

        # there is only one class
#         labels = torch.ones((records.shape[0],), dtype=torch.int64)

        # suppose all instances are not crowd
#         iscrowd = torch.zeros((records.shape[0],), dtype=torch.int64)

        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        # target['masks'] = None
        target['image_id'] = torch.tensor([index])
        target['area'] = area
#         target['iscrowd'] = iscrowd

        if self.transforms:
            sample = {
                'image': image,
                'bboxes': target['boxes'],
                'labels': labels
            }
            sample = self.transforms(**sample)
            image = sample['image']

            target['boxes'] = torch.stack(tuple(map(torch.tensor, zip(*sample['bboxes'])))).permute(1, 0)

        return image, target, path

    def __len__(self) -> int:
        return len(self.datas)




In [4]:
# Albumentations
def get_train_transform():
    return A.Compose([
        A.Resize(512, 512),
        A.RandomRotate90(),
        A.Flip(0.5),
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

def get_valid_transform():
    return A.Compose([
        A.RandomRotate90(),
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

In [5]:
def collate_fn(batch):
    return tuple(zip(*batch))

train_dataset = WheatDataset(datas, get_train_transform())


# split the dataset in train and test set
indices = torch.randperm(len(train_dataset)).tolist()

train_data_loader = DataLoader(
    train_dataset,
    batch_size=1,
    shuffle=False,
    num_workers=4,
    collate_fn=collate_fn
)

In [6]:

    
for images, targets, path in train_data_loader:
    print(path)
'success'

('/VOC2007/JPEGImages/img_4833.jpg',)
('/VOC2007/JPEGImages/img_773.jpg',)
('/VOC2007/JPEGImages/img_3184.jpg',)
('/VOC2007/JPEGImages/img_10962.jpg',)
('/VOC2007/JPEGImages/img_1793.jpg',)
('/VOC2007/JPEGImages/img_18480.jpg',)
('/VOC2007/JPEGImages/img_5293.jpg',)
('/VOC2007/JPEGImages/img_12813.jpg',)
('/VOC2007/JPEGImages/img_12807.jpg',)
('/VOC2007/JPEGImages/img_18494.jpg',)
('/VOC2007/JPEGImages/img_7490.jpg',)
('/VOC2007/JPEGImages/img_1787.jpg',)
('/VOC2007/JPEGImages/img_767.jpg',)
('/VOC2007/JPEGImages/img_3190.jpg',)
('/VOC2007/JPEGImages/img_11498.jpg',)
('/VOC2007/JPEGImages/img_15826.jpg',)
('/VOC2007/JPEGImages/img_001108.jpg',)
('/VOC2007/JPEGImages/img_997.jpg',)
('/VOC2007/JPEGImages/img_12191.jpg',)
('/VOC2007/JPEGImages/img_3806.jpg',)
('/VOC2007/JPEGImages/img_14286.jpg',)
('/VOC2007/JPEGImages/img_9495.jpg',)
('/VOC2007/JPEGImages/img_16491.jpg',)
('/VOC2007/JPEGImages/img_8947.jpg',)
('/VOC2007/JPEGImages/img_9481.jpg',)
('/VOC2007/JPEGImages/img_16485.jpg',)
('

('/VOC2007/JPEGImages/img_001524.jpg',)
('/VOC2007/JPEGImages/img_16903.jpg',)
('/VOC2007/JPEGImages/img_1383.jpg',)
('/VOC2007/JPEGImages/img_9907.jpg',)
('/VOC2007/JPEGImages/img_18090.jpg',)
('/VOC2007/JPEGImages/img_5683.jpg',)
('/VOC2007/JPEGImages/img_14872.jpg',)
('/VOC2007/JPEGImages/img_405.jpg',)
('/VOC2007/JPEGImages/img_001256.jpg',)
('/VOC2007/JPEGImages/img_14866.jpg',)
('/VOC2007/JPEGImages/img_7080.jpg',)
('/VOC2007/JPEGImages/img_1397.jpg',)
('/VOC2007/JPEGImages/img_9913.jpg',)
('/VOC2007/JPEGImages/img_3580.jpg',)
('/VOC2007/JPEGImages/img_377.jpg',)
('/VOC2007/JPEGImages/img_10396.jpg',)
('/VOC2007/JPEGImages/img_7916.jpg',)
('/VOC2007/JPEGImages/img_18912.jpg',)
('/VOC2007/JPEGImages/img_11088.jpg',)
('/VOC2007/JPEGImages/img_12581.jpg',)
('/VOC2007/JPEGImages/img_13847.jpg',)
('/VOC2007/JPEGImages/img_001518.jpg',)
('/VOC2007/JPEGImages/img_14696.jpg',)
('/VOC2007/JPEGImages/img_15588.jpg',)
('/VOC2007/JPEGImages/img_5867.jpg',)
('/VOC2007/JPEGImages/img_439.jpg',

('/VOC2007/JPEGImages/img_15990.jpg',)
('/VOC2007/JPEGImages/img_2310.jpg',)
('/VOC2007/JPEGImages/img_13339.jpg',)
('/VOC2007/JPEGImages/img_821.jpg',)
('/VOC2007/JPEGImages/img_14330.jpg',)
('/VOC2007/JPEGImages/img_5319.jpg',)
('/VOC2007/JPEGImages/img_12999.jpg',)
('/VOC2007/JPEGImages/img_2476.jpg',)
('/VOC2007/JPEGImages/img_3768.jpg',)
('/VOC2007/JPEGImages/img_4007.jpg',)
('/VOC2007/JPEGImages/img_16527.jpg',)
('/VOC2007/JPEGImages/img_11248.jpg',)
('/VOC2007/JPEGImages/img_10156.jpg',)
('/VOC2007/JPEGImages/img_9523.jpg',)
('/VOC2007/JPEGImages/img_6610.jpg',)
('/VOC2007/JPEGImages/img_17639.jpg',)
('/VOC2007/JPEGImages/img_3998.jpg',)
('/VOC2007/JPEGImages/img_5331.jpg',)
('/VOC2007/JPEGImages/img_13477.jpg',)
('/VOC2007/JPEGImages/img_15006.jpg',)
('/VOC2007/JPEGImages/img_7526.jpg',)
('/VOC2007/JPEGImages/img_11260.jpg',)
('/VOC2007/JPEGImages/img_17611.jpg',)
('/VOC2007/JPEGImages/img_6638.jpg',)
('/VOC2007/JPEGImages/img_18522.jpg',)
('/VOC2007/JPEGImages/img_17177.jpg',)

('/VOC2007/JPEGImages/img_001492.jpg',)
('/VOC2007/JPEGImages/img_13715.jpg',)
('/VOC2007/JPEGImages/img_3422.jpg',)
('/VOC2007/JPEGImages/img_8577.jpg',)
('/VOC2007/JPEGImages/img_7644.jpg',)
('/VOC2007/JPEGImages/img_18898.jpg',)
('/VOC2007/JPEGImages/img_11102.jpg',)
('/VOC2007/JPEGImages/img_1235.jpg',)
('/VOC2007/JPEGImages/img_18640.jpg',)
('/VOC2007/JPEGImages/img_18126.jpg',)
('/VOC2007/JPEGImages/img_17215.jpg',)
('/VOC2007/JPEGImages/img_11664.jpg',)
('/VOC2007/JPEGImages/img_19238.jpg',)
('/VOC2007/JPEGImages/img_7122.jpg',)
('/VOC2007/JPEGImages/img_3344.jpg',)
('/VOC2007/JPEGImages/img_13073.jpg',)
('/VOC2007/JPEGImages/img_5735.jpg',)
('/VOC2007/JPEGImages/img_12386.jpg',)
('/VOC2007/JPEGImages/img_5906.jpg',)
('/VOC2007/JPEGImages/img_11857.jpg',)
('/VOC2007/JPEGImages/img_10591.jpg',)
('/VOC2007/JPEGImages/img_7877.jpg',)
('/VOC2007/JPEGImages/img_17598.jpg',)
('/VOC2007/JPEGImages/img_9682.jpg',)
('/VOC2007/JPEGImages/img_16686.jpg',)
('/VOC2007/JPEGImages/img_18873.jp

('/VOC2007/JPEGImages/img_11464.jpg',)
('/VOC2007/JPEGImages/img_7322.jpg',)
('/VOC2007/JPEGImages/img_19038.jpg',)
('/VOC2007/JPEGImages/img_3144.jpg',)
('/VOC2007/JPEGImages/img_5535.jpg',)
('/VOC2007/JPEGImages/img_6014.jpg',)
('/VOC2007/JPEGImages/img_10752.jpg',)
('/VOC2007/JPEGImages/img_19010.jpg',)
('/VOC2007/JPEGImages/img_8039.jpg',)
('/VOC2007/JPEGImages/img_16323.jpg',)
('/VOC2007/JPEGImages/img_4603.jpg',)
('/VOC2007/JPEGImages/img_12145.jpg',)
('/VOC2007/JPEGImages/img_2272.jpg',)
('/VOC2007/JPEGImages/img_943.jpg',)
('/VOC2007/JPEGImages/img_14534.jpg',)
('/VOC2007/JPEGImages/img_14252.jpg',)
('/VOC2007/JPEGImages/img_4165.jpg',)
('/VOC2007/JPEGImages/img_8987.jpg',)
('/VOC2007/JPEGImages/img_10034.jpg',)
('/VOC2007/JPEGImages/img_9441.jpg',)
('/VOC2007/JPEGImages/img_18468.jpg',)
('/VOC2007/JPEGImages/img_6772.jpg',)
('/VOC2007/JPEGImages/img_7478.jpg',)
('/VOC2007/JPEGImages/img_10020.jpg',)
('/VOC2007/JPEGImages/img_1009.jpg',)
('/VOC2007/JPEGImages/img_8993.jpg',)
('

...

('/VOC2007/JPEGImages/img_2094.jpg',)
('/VOC2007/JPEGImages/img_11872.jpg',)
('/VOC2007/JPEGImages/img_555.jpg',)
('/VOC2007/JPEGImages/img_14922.jpg',)
('/VOC2007/JPEGImages/img_13095.jpg',)
('/VOC2007/JPEGImages/img_11682.jpg',)
('/VOC2007/JPEGImages/img_16853.jpg',)
('/VOC2007/JPEGImages/img_17595.jpg',)
('/VOC2007/JPEGImages/img_233.jpg',)
('/VOC2007/JPEGImages/img_2902.jpg',)
('/VOC2007/JPEGImages/img_15382.jpg',)
('/VOC2007/JPEGImages/img_14088.jpg',)
('/VOC2007/JPEGImages/img_2916.jpg',)
('/VOC2007/JPEGImages/img_8585.jpg',)
('/VOC2007/JPEGImages/img_9843.jpg',)
('/VOC2007/JPEGImages/img_17581.jpg',)
('/VOC2007/JPEGImages/img_10588.jpg',)
('/VOC2007/JPEGImages/img_11696.jpg',)
('/VOC2007/JPEGImages/img_14936.jpg',)
('/VOC2007/JPEGImages/img_001306.jpg',)
('/VOC2007/JPEGImages/img_2043.jpg',)
('/VOC2007/JPEGImages/img_14705.jpg',)
('/VOC2007/JPEGImages/img_4432.jpg',)
('/VOC2007/JPEGImages/img_12374.jpg',)
('/VOC2007/JPEGImages/img_19221.jpg',)
('/VOC2007/JPEGImages/img_8208.jpg'

('/VOC2007/JPEGImages/img_3956.jpg',)
('/VOC2007/JPEGImages/img_2490.jpg',)
('/VOC2007/JPEGImages/img_179.jpg',)
('/VOC2007/JPEGImages/img_4787.jpg',)
('/VOC2007/JPEGImages/img_15976.jpg',)
('/VOC2007/JPEGImages/img_5499.jpg',)
('/VOC2007/JPEGImages/img_001058.jpg',)
('/VOC2007/JPEGImages/img_6190.jpg',)
('/VOC2007/JPEGImages/img_19194.jpg',)
('/VOC2007/JPEGImages/img_6184.jpg',)
('/VOC2007/JPEGImages/img_1833.jpg',)
('/VOC2007/JPEGImages/img_19180.jpg',)
('/VOC2007/JPEGImages/img_4793.jpg',)
('/VOC2007/JPEGImages/img_15962.jpg',)
('/VOC2007/JPEGImages/img_2484.jpg',)
('/VOC2007/JPEGImages/img_8817.jpg',)
('/VOC2007/JPEGImages/img_12943.jpg',)
('/VOC2007/JPEGImages/img_13485.jpg',)
('/VOC2007/JPEGImages/img_145.jpg',)
('/VOC2007/JPEGImages/img_17185.jpg',)
('/VOC2007/JPEGImages/img_8181.jpg',)
('/VOC2007/JPEGImages/img_15792.jpg',)
('/VOC2007/JPEGImages/img_623.jpg',)
('/VOC2007/JPEGImages/img_4963.jpg',)
('/VOC2007/JPEGImages/img_15786.jpg',)
('/VOC2007/JPEGImages/img_001070.jpg',)
('

('/VOC2007/JPEGImages/img_14880.jpg',)
('/VOC2007/JPEGImages/img_12229.jpg',)
('/VOC2007/JPEGImages/img_3200.jpg',)
('/VOC2007/JPEGImages/img_15552.jpg',)
('/VOC2007/JPEGImages/img_11734.jpg',)
('/VOC2007/JPEGImages/img_8341.jpg',)
('/VOC2007/JPEGImages/img_17345.jpg',)
('/VOC2007/JPEGImages/img_1365.jpg',)
('/VOC2007/JPEGImages/img_9739.jpg',)
('/VOC2007/JPEGImages/img_18710.jpg',)
('/VOC2007/JPEGImages/img_17423.jpg',)
('/VOC2007/JPEGImages/img_8427.jpg',)
('/VOC2007/JPEGImages/img_7714.jpg',)
('/VOC2007/JPEGImages/img_385.jpg',)
('/VOC2007/JPEGImages/img_3572.jpg',)
('/VOC2007/JPEGImages/img_5103.jpg',)
('/VOC2007/JPEGImages/img_13645.jpg',)
('/VOC2007/JPEGImages/img_10364.jpg',)
('/VOC2007/JPEGImages/img_6422.jpg',)
('/VOC2007/JPEGImages/img_18738.jpg',)
('/VOC2007/JPEGImages/img_9711.jpg',)
('/VOC2007/JPEGImages/img_16715.jpg',)
('/VOC2007/JPEGImages/img_19426.jpg',)
('/VOC2007/JPEGImages/img_12573.jpg',)
('/VOC2007/JPEGImages/img_4235.jpg',)
('/VOC2007/JPEGImages/img_14102.jpg',)

('/VOC2007/JPEGImages/img_18328.jpg',)
('/VOC2007/JPEGImages/img_6032.jpg',)
('/VOC2007/JPEGImages/img_10774.jpg',)
('/VOC2007/JPEGImages/img_2254.jpg',)
('/VOC2007/JPEGImages/img_965.jpg',)
('/VOC2007/JPEGImages/img_12163.jpg',)
('/VOC2007/JPEGImages/img_12605.jpg',)
('/VOC2007/JPEGImages/img_000582.jpg',)
('/VOC2007/JPEGImages/img_4143.jpg',)
('/VOC2007/JPEGImages/img_14274.jpg',)
('/VOC2007/JPEGImages/img_2532.jpg',)
('/VOC2007/JPEGImages/img_10012.jpg',)
('/VOC2007/JPEGImages/img_9467.jpg',)
('/VOC2007/JPEGImages/img_6754.jpg',)
('/VOC2007/JPEGImages/img_8779.jpg',)
('/VOC2007/JPEGImages/img_10006.jpg',)
('/VOC2007/JPEGImages/img_9473.jpg',)
('/VOC2007/JPEGImages/img_6740.jpg',)
('/VOC2007/JPEGImages/img_17769.jpg',)
('/VOC2007/JPEGImages/img_6998.jpg',)
('/VOC2007/JPEGImages/img_12611.jpg',)
('/VOC2007/JPEGImages/img_4157.jpg',)
('/VOC2007/JPEGImages/img_14260.jpg',)
('/VOC2007/JPEGImages/img_5249.jpg',)
('/VOC2007/JPEGImages/img_2526.jpg',)
('/VOC2007/JPEGImages/img_2240.jpg',)
(

'success'

In [7]:
'success'

'success'